Noisy and erroneous data is a major bottleneck in analytics. Data cleaning and repairing account for about 60% of the work of data scientists. To address this bottleneck, we recently introduced HoloClean, a semi-automated data repairing framework that relies on statistical learning and inference to repair errors in structured data. In HoloClean, we build upon the paradigm of weak supervision and demonstrate how to leverage diverse signals, including user-defined heuristic rules (such as generalized data integrity constraints) and external dictionaries, to repair erroneous data. 

In this post, we walk through the process of implementing Holoclean, by creating a simple end-to-end example.

# Setup

Firstly, we import all the module from Holoclean that we will use.

In [1]:
from holoclean.holoclean import HoloClean, Session
from holoclean.errordetection.errordetector import ErrorDetectors
from holoclean.featurization.featurizer import SignalInit, SignalCooccur, SignalDC
from holoclean.featurization.featurizer import Featurizer
from holoclean.learning.softmax import SoftMax
from holoclean.learning.accuracy import Accuracy
import time

##   Initialization
In this part, we create the Holoclean and Session object that we will use for this example.

In [2]:
holo_obj = HoloClean(mysql_driver = "../holoclean/lib/mysql-connector-java-5.1.44-bin.jar" )
session = Session("Session", holo_obj)
        

/Users/joshmcgrath/anaconda2/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1569: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input and DC from file
Test data and the Denial Constraints will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
dataset = "../datasets/hospital1k/hospital_dataset.csv"

denial_constraints = "../datasets/hospital1k/hospital_constraints.txt"

ground_truth = "../datasets/hospital1k/groundtruth.csv"

# Ingesting Dataset and Denial Constraints

session.ingest_dataset(dataset)



session.denial_constraints(denial_constraints)

sql = holo_obj.dataengine.get_table_to_dataframe("Init", session.dataset)


## Error Detection
In this part, we create the error detection. The output of this part is the C_dk table that contains all the noisy cells and the C_Clean table that contains the clean cells

In [ ]:
t0 = time.time()
err_detector = ErrorDetectors(session.Denial_constraints, holo_obj, session.dataset,detection_type=None)
session.add_error_detector(err_detector)
session.ds_detect_errors()

t1 = time.time()
total = t1 - t0


## Domain Pruning
In this part, we prune the domain. The output of this part is the possible_values tables that contains all the possible values for each cell

In [5]:
t0 = time.time()
pruning_threshold = 0.5
session.ds_domain_pruning(pruning_threshold)

t1 = time.time()
total = t1 - t0
holo_obj.logger.info('domain pruning time: '+str(total)+'\n')
print 'domain pruning time: '+str(total)+'\n'

print 'Possible_values_clean'
sql = holo_obj.dataengine.get_table_to_dataframe("Possible_values_clean", session.dataset)
sql.show()

print 'Possible values dk'
sql = holo_obj.dataengine.get_table_to_dataframe("Possible_values_dk", session.dataset)
sql.show()

domain pruning time: 37.3310968876

Possible_values_clean
+----+---+-----------+--------------------+--------+---------+
| vid|tid|  attr_name|            attr_val|observed|domain_id|
+----+---+-----------+--------------------+--------+---------+
| 892|496|  Condition|        Heart Attack|       1|        1|
| 893|496|MeasureCode|               AMI-x|       1|        1|
| 893|496|MeasureCode|               AMI-5|       0|        2|
| 238|141|       City|             LUVERNE|       1|        1|
|1615|854|PhoneNumber|          2059216200|       1|        1|
| 692|399|       City|          PRATTVILLE|       1|        1|
|1132|630|      State|                  AL|       1|        1|
| 485|288|  Condition|Surgical Infectio...|       1|        1|
| 894|496|MeasureName|Heart Attack Pati...|       1|        1|
| 239|141|   Stateavg|       AL_SCIP-INF-4|       1|        1|
|1616|855|      State|                  AL|       1|        1|
|   1|  1|      State|                  AL|       1|        

# Featurization

In this part, we implement the featurization module of holoclean. We choose the signals that we want to use and the output of this part is the featurization table that contains the factors that we will use

## Feature Signals

In [6]:
t0 = time.time()
initial_value_signal = SignalInit(session.Denial_constraints, holo_obj.dataengine,
                              session.dataset)
session.add_featurizer(initial_value_signal )
statistics_signal = SignalCooccur(session.Denial_constraints, holo_obj.dataengine,
                              session.dataset )
session.add_featurizer(statistics_signal)
dc_signal = SignalDC(session.Denial_constraints, holo_obj.dataengine, session.dataset,
                 holo_obj.spark_session)
session.add_featurizer(dc_signal)
t1 = time.time()
total = t1 - t0
print "Feature Signal Time:", total

Feature Signal Time: 0.000611066818237


We use the signals that we choose in the previous step. The output of this part is the featurization table that contains the factors that we will use in the next step.

In [7]:
t0 = time.time()
session.ds_featurize(clean = 1)

t1 = time.time()

total = t1 - t0

holo_obj.logger.info('featurization time: '+str(total)+'\n')
print 'featurization time: '+str(total)+'\n'

featurization time: 14.0898318291



#  Learning
We create the X-tensor from the feature_clean table and run softmax on it, then we save results to MySQL and output the accuracy

In [8]:
t0 = time.time()
soft = SoftMax(holo_obj.dataengine, session.dataset, holo_obj,
                       session.X_training)

soft.logreg()
t1 = time.time()
total = t1 - t0

print 'time for training model: '+str(total)+'\n'


time for training model: 112.9060359



In this part, we use the new weight, to learn the probabilities for each value for the cells


In [9]:
t0 = time.time()
session.ds_featurize(clean = 0)
t1 = time.time()
total = t1 - t0
print 'time for test featurization: ' + str(total) + '\n'

Y = soft.predict(soft.model, session.X_testing, soft.setupMask(0, session.N, session.L))
t1 = time.time()
total = t1 - t0
print 'time for inference: ', total
soft.save_prediction(Y)

print 'Inferred values for dk cells'
sql = holo_obj.dataengine.get_table_to_dataframe("Inferred_values", session.dataset)
sql.show()


flattening = 0

acc = Accuracy(holo_obj.dataengine, ground_truth, session.dataset,
               holo_obj.spark_session)
acc.accuracy_calculation(flattening)


endtime = time.time()
print 'total time: ', endtime - start_time

time for test featurization: 11.1193227768

time for inference:  11.4453318119
Inferred values for dk cells
+------------------+----+-----------+--------------------+---+---------+
|       probability| vid|  attr_name|            attr_val|tid|domain_id|
+------------------+----+-----------+--------------------+---+---------+
|0.5953319072723389| 406|PhoneNumber|          2565938310| 63|        1|
|               1.0|   3|PhoneNumber|          2053258100|  1|        1|
|               1.0| 307|  Condition|        Heart Attack| 49|        1|
|               1.0|1161|MeasureCode|               PN-5C|184|        1|
|               1.0| 196|    ZipCode|               36302| 31|        1|
|0.5996459126472473|  17|       City|          BIRMINGHAM|  4|        1|
|0.5955309867858887| 442|PhoneNumber|          2565938310| 69|        1|
|               1.0|  48|      State|                  AL|  9|        1|
|               1.0|  58|PhoneNumber|          2563864556| 10|        1|
|0.5283018946647